<a href="https://colab.research.google.com/github/mhirschberg/py_demo/blob/main/FreeAImode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Install required dependencies
# - requests: For making HTTP calls to Bright Data APIs
!pip install -q requests
# Import required libraries
import requests
import json
import time
from rich.console import Console
from rich.markdown import Markdown

In [36]:
def poll_and_download_snapshot(snapshot_id, token):
    monitor_url = f"https://api.brightdata.com/datasets/v3/progress/{snapshot_id}"
    download_url = f"https://api.brightdata.com/datasets/v3/snapshot/{snapshot_id}"

    headers = {"Authorization": f"Bearer {token}"}

    while True:
        response = requests.get(monitor_url, headers=headers)
        status = response.json().get("status")

        if status == "ready":
            download_response = requests.get(download_url, headers=headers)
            download_data = download_response.json()

            # Check if download is actually ready or still building
            if download_data.get("status") == "building":
                time.sleep(10)
                continue

            print(download_response.text)
            return download_data

        time.sleep(10)

def format_response(data):
    if isinstance(data, str):
        import json
        data = json.loads(data)

    console = Console()

    console.print("[bold cyan]PROMPT:[/bold cyan]")
    console.print(data.get('prompt', ''))
    print()

    console.print("[bold cyan]ANSWER:[/bold cyan]")

    answer = data.get('answer_text_markdown', data.get('answer_text', ''))
    md = Markdown(answer)
    console.print(md)
    print()
    console.print("[bold cyan]CITATIONS:[/bold cyan]")

    for i, citation in enumerate(data.get('citations', []), 1):
        console.print(f"[bold]{i}.[/bold] {citation.get('title', 'N/A')} - [dim]{citation.get('domain', 'N/A')}[/dim]")
        console.print(f"   [link={citation.get('url', 'N/A')}]{citation.get('url', 'N/A')}[/link]")
        console.print()

In [11]:
#@title 👉 Enter Your Prompt Here
#@markdown ---
#@markdown Enter your prompt below, then run the rest of the notebook.
#@markdown ---

MY_PROMPT = "What day is it today?" #@param {type:"string"}
MY_COUNTRY = "US" #@param {type:"string"}

# ============================================
# API credentials (from Colab secrets)
# ============================================
from google.colab import userdata
import os

# API KEY
os.environ['BRIGHTDATA_API_TOKEN'] = userdata.get('BRIGHTDATA_API_TOKEN')
BRIGHTDATA_API_TOKEN = os.environ.get("BRIGHTDATA_API_TOKEN")

print("Configuration loaded:")
print(f"  Prompt: {MY_PROMPT}")
print(f"  Country: {MY_COUNTRY}")
print(f"  Bright Data API: {'configured' if BRIGHTDATA_API_TOKEN else 'X missing'}")

Configuration loaded:
  Prompt: What day is it today?
  Country: US
  Bright Data API: ✓ configured


In [35]:
headers = {
    "Authorization": f"Bearer {BRIGHTDATA_API_TOKEN}",
    "Content-Type": "application/json",
}

data = json.dumps({
    "input": [{"url": "https://google.com/aimode", "prompt": MY_PROMPT, "country": MY_COUNTRY}],
})

response = requests.post(
    "https://api.brightdata.com/datasets/v3/scrape?dataset_id=gd_mcswdt6z2elth3zqr2&custom_output_fields=prompt%2Canswer_text%2Ccitations%2Canswer_text_markdown%2Ctimestamp&notify=false&include_errors=true",
    headers=headers,
    data=data
)

bd_answer = (response.json())

if 'snapshot_id' in bd_answer:
    snapshot_id = bd_answer['snapshot_id']
    bd_answer = poll_and_download_snapshot(snapshot_id, BRIGHTDATA_API_TOKEN)

format_response(bd_answer)

PROMPT:

What day is it today?

ANSWER:

Today is                                                                                                           

Friday, February 13, 2026 .                                                                                        

Observances for Friday, Feb 13                                                                                     

Several holidays and awareness events are celebrated today:                                                        

 • World Radio Day : An international day established by UNESCO to celebrate radio as a medium.                    
 • Galentine’s Day : A day for celebrating female friendships, typically observed the day before Valentine's Day.  
 • National Tortellini Day : A food holiday dedicated to the ring-shaped pasta.                                    
 • National Apology Day : Observed in Australia to commemorate the anniversary of the 2008 apology to the Stolen   
   Generations.                                                                                                    
 • Blame Someone Else Day : A humorous holiday traditionally observed on any Friday the 13th.                      

Historical Events on this Date                                                                                     

 • 1945 : Allied forces began the three-day bombing of Dresden, Germany, during World War II.                      
 • 1980 : The 13th Winter Olympics opened in Lake Placid, New York.                                                
 • 1996 : The musical Rent  premiered off-Broadway.                                                                

Would you like to know about any upcoming events  this weekend or more historical facts  for this date?

CITATIONS:

...

]8;id=438465;https://wtop.com/back-in-the-day/2026/02/today-in-history-february-13-the-world-war-ii-bombing-of-dresden/#:~:text=The%20Associated%20Press,three%20weeks%20after%20Larson's%20death.\https://wtop.com/back-in-the-day/2026/02/today-in-history-february-13-the-world-war-ii-bombing-of-dresden/#:~:]8;;\]8;id=858709;https://wtop.com/back-in-the-day/2026/02/today-in-history-february-13-the-world-war-ii-bombing-of-dresden/#:~:text=The%20Associated%20Press,three%20weeks%20after%20Larson's%20death.\te]8;;\
]8;id=858709;https://wtop.com/back-in-the-day/2026/02/today-in-history-february-13-the-world-war-ii-bombing-of-dresden/#:~:text=The%20Associated%20Press,three%20weeks%20after%20Larson's%20death.\xt]8;;\]8;id=438465;https://wtop.com/back-in-the-day/2026/02/today-in-history-february-13-the-world-war-ii-bombing-of-dresden/#:~:text=The%20Associated%20Press,three%20weeks%20after%20Larson's%20death.\=]8;;\]8;id=659077;https://wtop.com/back-in-the-day/2026/02/today-in-history-february-13-the-world-war-ii-bombing-of-dresden/#:~:text=The%20Associated%20Press,three%20weeks%20after%20Larson's%20death.\The]8;;\]8;id=438465;https://wtop.com/back-in-the-day/2026/02/today-in-history-february-13-the-world-war-ii-bombing-of-dresden/#:~:text=The%20Associated%20Press,three%20weeks%20after%20Larson's%20death.\%20Associated%20Press,three%20weeks%20after%20Larson]8;;\]8;id=104283;https://wtop.com/back-in-the-day/2026/02/today-in-history-february-13-the-world-war-ii-bombing-of-dresden/#:~:text=The%20Associated%20Press,three%20weeks%20after%20Larson's%20death.\'s%20death.]8;;\

10. National Today - https://nationaltoday.com

]8;id=988407;https://nationaltoday.com/today/\https://nationaltoday.com/today/]8;;\

11. There is a Day for That! - https://www.thereisadayforthat.com

]8;id=777373;https://www.thereisadayforthat.com/\https://www.thereisadayforthat.com/]8;;\